In [0]:
%pip install dspy-ai==2.4.10
%pip install textstat
%pip install --upgrade openai
%pip install 'crewai[tools]'
%pip install rich
%pip install duckduckgo-search langchain-community
%pip install --upgrade openai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.

*** WARNING: max output size exceeded, skipping output. ***

n /local_disk0/.ephemeral_nfs/envs/pythonEnv-09496ab9-f6b6-4f29-97ac-ba837dabb231/lib/python3.10/site-packages (from boto3<2.0.0,>=1.34.0->cohere<6.0,>=5.3->embedchain<0.2.0,>=0.1.114->crewai[tools]) (0.10.2)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated p

In [0]:
%restart_python

In [0]:
import dspy
API_BASE = "https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints"
API_KEY = dbutils.secrets.get("oetrta","sg-aztoken")
external_model = dspy.Databricks(model='sg-external',model_type='chat', api_key = API_KEY, api_base = API_BASE,max_tokens=2000,temperature=0.7)
dspy.configure(lm=external_model)

In [0]:
class SocialMediaPostGenerator(dspy.Signature):
    """You are a social media assistant who generates an engaging social media post with hashtags about the user's experience at the Databricks Data & AI World Tour 2024 (DAIWT) in Atlanta. This is a tech conference about Data and AI. Call out sessions where the local time overlaps with the user's post and assume that the user is one of the sessions if overlap happens between local time and session time. Each session is atleast 40 minutes. If there are multiple sessions that overlap, always choose AI sessions. If there are no AI sessions, choose based on current session.Give all posts a very positive spin to help it go viral. Ongoing sessions take priority, unless the user specifically calls out other sessions or topics"""
    local_time = dspy.InputField()
    user_post = dspy.InputField()
    user_role = dspy.InputField(desc="User's role in the conference. Either attendee or organizer or presenter")
    agenda = dspy.InputField()
    social_media_site = dspy.InputField()
    rationale = dspy.OutputField(desc="Reasoning behind the post content and hashtags")
    current_session=dspy.OutputField(desc="Current session that the user could be in. If there isn't a match, return None")
    post = dspy.OutputField(desc="Engaging social media post with hashtags. Pay specific attention to any current session the user is in.This is the ongoing session.")


In [0]:
class EngagingSocialMediaPost(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.ChainOfThought(SocialMediaPostGenerator)

    def forward(self, local_time, user_post, user_role, agenda, social_media_site):
        return self.generator(
            local_time=local_time,
            user_post=user_post,
            user_role=user_role,
            agenda=agenda,
            social_media_site=social_media_site
        )

In [0]:
import json
with open("agenda.json", "r") as file:
    agenda = json.load(file)

agenda = json.dumps(agenda)

In [0]:
from datetime import datetime
examples = [
    dspy.Example(
        local_time="09:30 AM",
        user_post="Excited for my presentation today!",
        user_role="presenter",
        agenda=agenda,
        social_media_site="LinkedIn",
        rationale="The post should be professional and optimistic, highlighting the upcoming presentation. We'll use hashtags related to professional growth and presentations. The time suggests it's morning, so we can incorporate that.",
        post="Good morning, LinkedIn! ☀️ Kicking off a productive day with a team meeting, followed by an exciting client presentation this afternoon. Ready to showcase our latest innovations! #ProfessionalGrowth #ClientPresentation #InnovationInAction"
    ),
    dspy.Example(
        local_time="1:45 PM",
        user_post="Looking forward to my presentation!",
        user_role="presenter",
        agenda=agenda,
        social_media_site="LinkedIn",
    rationale="The post should be professional yet engaging, suitable for Instagram. We'll focus on the upcoming presentation, incorporating the user's role as a presenter. The time suggests it's just before the presentation, so we'll emphasize preparation and excitement. We'll use relevant hashtags to increase visibility and engagement.",
    post="Pre-presentation butterflies! 🦋 Just 15 minutes until I take the stage to share our latest innovations with our valued clients. Months of hard work have led to this moment. Excited to showcase what our amazing team has accomplished! 💼✨ #ProfessionalGrowth #PublicSpeaking #InnovationPresentation #ReadyToInspire")
]

# Create and compile the model
post_generator = EngagingSocialMediaPost()
post_generator.generator.demos = examples

# Function to generate post
def generate_social_media_post(local_time, user_post, user_role, agenda, social_media_site):
    # results = []
    result = post_generator(
        local_time=local_time,
        user_post=user_post,
        user_role = user_role,
        agenda=days_agenda,
        social_media_site=social_media_site
    )
    # results.append(result)
    return result



In [0]:
# Example usage
# local_time = datetime.now().strftime("%I:%M %p")
local_time = "2:10 PM EDT"
user_post = "This is a great presentation on llmops! fun!"
user_role = "attendee"
days_agenda = agenda
social_media_site = "LinkedIn"

In [0]:
test = generate_social_media_post(local_time, user_post,user_role, days_agenda, social_media_site)

In [0]:
test.post

"Just wrapped up an insightful session on LLMOps! 🚀 The fascinating world of Large Language Model Operations is truly evolving, and I’m thrilled to learn about building composable AI systems with Databricks. It's amazing to see how these innovations are shaping the future of data and AI! 🌟 #DataAndAI #LLMOps #Innovation #TechConference #Databricks\n\n---\n\nFeel free to let me know if you need further assistance or another post!"

In [0]:
test.post.split('\n')[0]

"Just wrapped up an insightful session on LLMOps! 🚀 The fascinating world of Large Language Model Operations is truly evolving, and I’m thrilled to learn about building composable AI systems with Databricks. It's amazing to see how these innovations are shaping the future of data and AI! 🌟 #DataAndAI #LLMOps #Innovation #TechConference #Databricks"

In [0]:
import dspy

class ImgGenSignature(dspy.Signature):
  user_post = dspy.InputField(desc="the social media post the user wants to make")
  negative_prompt = dspy.InputField(desc='include this statement to the prompt to avoid generating incorrect images')
  extracted_topics = dspy.OutputField(desc="the extracted topics from the user's post relevant to Data and AI. must be specific to databricks. include mosaic ai if user is interested in AI")
  flux_prompt = dspy.OutputField(desc="the prompt to be used for generating the image worthy of sharing in social media using flux1 image generation models.Focus on the ambience.")
  

class SocialMediaProcessor(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prompt_generator = dspy.ChainOfThought(ImgGenSignature)

    def forward(self, user_post, negative_prompt):
        result = self.prompt_generator(user_post=user_post, negative_prompt=negative_prompt)
        return result
        

In [0]:

# Usage example
processor = SocialMediaProcessor()
result = processor(
    user_post=test.post.split('\n')[0],
    negative_prompt="Do not include any text in the image or specific references to events.Do not include any words or any reference to databricks world tour"
)

In [0]:
result

Prediction(
    rationale="produce the flux_prompt. We need to first identify the key topics that are relevant to the user's post. The user is discussing LLMOps and the evolution of Large Language Model Operations, which relates to the field of Data and AI. Additionally, the user specifically mentions Databricks, which is a key aspect of their interest in AI and data innovations. \n\nFrom the user post, we can extract the following topics:\n1. LLMOps (Large Language Model Operations)\n2. Composable AI systems\n3. The role of Databricks in AI and data innovation\n\nNext, we need to formulate the flux_prompt. This prompt should encapsulate the themes of innovation and technology while ensuring it aligns with the ambience suitable for social media sharing. Since the negative prompt instructs us to avoid any text or specific event references, we will focus on creating an engaging visual representation that reflects the excitement and evolution in the field of Data and AI, particularly with

In [0]:
# external_model_pplx = dspy.Databricks(model='sg-pplx',model_type='chat', api_key = API_KEY, api_base = API_BASE,max_tokens=2000,temperature=0.7)
# dspy.configure(lm=external_model_pplx)

In [0]:
# class SearchContent(dspy.Signature):
#   search_query= dspy.InputField()
#   answer=dspy.OutputField(desc="Should contain links to databricks website")

# aggregate_result = dspy.ChainOfThought(SearchContent)
# search_query = f"Give me top 3 blogs with links from databricks website in the last 2 months related to the following topics {result.extracted_topics}. Results must include links"
# print(search_query)
# pred = aggregate_result(search_query=search_query)

In [0]:
search_query = f"databricks blogs and videos related to the following databricks topics {result.extracted_topics}. "
print(search_query)

databricks blogs and videos related to the following databricks topics 1. LLMOps (Large Language Model Operations)
2. Composable AI systems
3. Databricks and its innovations in AI. 


In [0]:
# search_query = "databricks blogs and videos related to the following databricks topics Data foundation, AI, Knowledge sharing, Innovation, Databricks."

In [0]:
search_query

'databricks blogs and videos related to the following databricks topics 1. LLMOps (Large Language Model Operations)\n2. Composable AI systems\n3. Databricks and its innovations in AI. '

In [0]:
# messages =[
#     {
#       "role": "user",
#       "content": "Hello!"
#     },
#     {
#       "role": "assistant",
#       "content": "Hello! How can I assist you today?"
#     },
#     {
#       "role": "user",
#       "content": search_query if search_query else "None",
#     }]
# from openai import OpenAI
# from databricks.sdk import WorkspaceClient
# import time

# w = WorkspaceClient()
# ENDPOINT_NAME = "sg-pplx"

# client = OpenAI(
#     api_key=w.tokens.create(
#         comment=f"sdk-{time.time_ns()}", lifetime_seconds=120
#     ).token_value,
#     base_url=f"{w.config.host}/serving-endpoints",
# )

# response = client.chat.completions.create(
#     model=ENDPOINT_NAME,
#     messages=messages,
#     max_tokens=2000,
#     temperature=0.1
# )

In [0]:
# response.choices[0].message.content

In [0]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
sr = search.invoke(search_query)

In [0]:
messages =[
    {
      "role": "user",
      "content": "Hello!"
    },
    {
      "role": "assistant",
      "content": "Hello! How can I assist you today?"
    },
    {
      "role": "user",
      "content": f"Extract all the links from this text, produce a JSON array. Only output this JSON and nothing else: {sr}",
    }]

In [0]:
ENDPOINT_NAME = "sg-external"
from openai import OpenAI
from databricks.sdk import WorkspaceClient
import time
w = WorkspaceClient()
client = OpenAI(
    api_key=w.tokens.create(
        comment=f"sdk-{time.time_ns()}", lifetime_seconds=120
    ).token_value,
    base_url=f"{w.config.host}/serving-endpoints",
)

response = client.chat.completions.create(
    model=ENDPOINT_NAME,
    messages=messages,
    max_tokens=2000,
    temperature=0.1
)

In [0]:
from rich import print
import json
json_response = response.choices[0].message.content

In [0]:

json.loads(json_response[json_response.find("["):json_response.find("]")+1].strip('\n'))

['https://www.databricks.com/blog/big-book-mlops-updated-generative-ai',
 'https://www.databricks.com/blog/building-custom-genai-llms-and-beyond',
 'https://community.databricks.com/t5/technical-blog/introducing-mlops-gym-your-practical-guide-to-mlops-on/ba-p/62576',
 'https://www.databricks.com/blog/category/engineering/data-science-machine-learning']